# Eritrea Twitter


name: Coleman Shepard <br>

### Twitter Authentication Information

In [ ]:
# Insert your consumer key and secret inside the quotes below
consumer_key = ""
consumer_secret = ""

# Insert your access key and secret inside the quotes below
access_token_key = ""
access_token_secret = ""

print("Consumer and access keys and secrets have been successfully added")


### Import Modules

In [ ]:
# Example: Search for and print tweets using the 'search/tweets' endpoint 

# This line allows us to access the TwitterAPI library
from TwitterAPI import TwitterAPI
import pprint
import psycopg2
import time

### Connect and Create a Cursor for a Postgres Database

Instead of writing the data collected from Twitter to a csv file, I chose to write the streamed data to a postgres database using the psycopg2 module in Python. This way I do not have to worry about any cocurrency issues when I query data from the database if data is still being writing to it. This also allows multiple users/scripts to write to the database which could be useful with multiple people working on the same project. The database is locally hosted (At the moment), so passwords and other information are close to default. If the database was hosted somewhere, I would explore the security measures that need to be in place. 

In [ ]:
# Creates a connection to a database in Postgres (Local)
twitter_connection = psycopg2.connect(user = "tweet",
                                  password = "ggez",
                                  host = "127.0.0.1",
                                  port = "5433",
                                  database = "twitter")

# Needed for connection
cursor = twitter_connection.cursor()

# Print PostgreSQL Connection properties
print (twitter_connection.get_dsn_parameters(),"\n")

## Create a table in the database

A database called twitter was created through PgAdmin, but I wanted to practice creating tables with desired schemas in Python. At this point, I just assigned a Primary Key which will correspond to a counter variable running the stream for tweets. Right now, I'm not sure if this is the best way to go about this, especially if I were to index it. A clustered index around hashtags may prove to be better for my goal. At the moment, I'm just worried about getting going with Postgres & Python. I may make alterations down the line to optimize the retrieval of information. 

### Schema Creation

In [ ]:
# Schema for Postgres Table Information Collection
Table = """CREATE TABLE twitter_t
                (ID INT PRIMARY KEY NOT NULL,
                TIME REAL,
                NAME TEXT[],
                FULL_TEXT TEXT[],
                HASHTAG TEXT[],
                COORDINATE TEXT,
                GEO TEXT,
                PLACE TEXT,
                FAV_COUNT INT,
                QUOTE_COUNT INT,
                REPLY_COUNT INT,
                RETWEET_COUNT INT,
                STATUS_COUNT INT,
                SCREEN_NAME TEXT,
                FRIENDS_COUNT INT);"""

### Execute Table Command 

In [ ]:
# Execute CREATE TABLE command
cursor.execute(Table)

# Commit the action to the database
twitter_connection.commit()

# Close the cursor
twitter_connection.close()
print("Table created successfully in PostgreSQL ")

## Twitter Script

This is the script I use to gather Twitter data and write it out to a record in my database. 

In [ ]:

# Creates a connection to a database in Postgres (Local)
twitter_connection = psycopg2.connect(user = "tweet",
                                  password = "ggez",
                                  host = "127.0.0.1",
                                  port = "5433",
                                  database = "twitter")
# Needed for connection
cursor = twitter_connection.cursor()


print("Starting TwitterAPI")
api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret,)

# Look for keywords in tweets
keywords = "eritrea","TPLF", "AbiyAhmed", "IsaiasAfewerki", "PFDJ","EPRDF","FreeEritrea", "IStandWithEritrea","#IStandWithEritrea","#Eritrea","HandsOffEritrea", "#HandsOffEritrea",

while True:
    try:
        print("Setting up filter request")
        # Notice we can add keyword filters (called 'track' term in Twitter)
        r = api.request('statuses/filter', {'track': keywords})

        # Count for database entry (Mistake)
        count = 4666


        print("Begin capturing tweets")
        for tweet in r:

            hashtags = [i['text'] + " "  for i in tweet['entities']['hashtags']]

            coords = tweet['coordinates']
        #     print(hashtags)
        #     print("ID",tweet['id'])
        #     print("TIME", tweet['created_at'])
        #     print("NAME",tweet['user']['name'])
        #     print("Full Text",tweet['text'])
        #     print("Hashtag",tweet['entities']['hashtags'])
            print("Coordinates",tweet['coordinates'])
            print("Geo",tweet['geo'])
            print("Place",tweet['place'])
        #     print("Fav_Count",tweet['favorite_count'])
        #     print("Quote_Count",tweet['quote_count'])
        #     print("Reply_Count",tweet['reply_count'])
        #     print("Retweet", tweet['retweet_count'])
        #     print("Statuses_Count", tweet['user']['statuses_count'])
            print("Screen_Name", tweet['user']['screen_name'])
            print("Friends_Count", tweet['user']['friends_count'])

            if tweet["retweeted"] == True:
                print("TRUE")

            if 'extended_tweet' in tweet:

                text = tweet['extended_tweet']['full_text']
            else:
                text = tweet['text']

            if 'coordinates' in tweet and tweet['coordinates'] != None:
                coords = str((tweet['coordinates']["coordinates"][0],tweet['coordinates']["coordinates"][1])) # Add the tweet coordinates to the list
                print(tweet['coordinates'])
            else:
                coords = tweet['coordinates']# If the tweet does not have coordinates then skip it


            if tweet['place'] != None:
                place = str(tweet['place']["full_name"]) # Add the tweet coordinates to the list

            else:
                place = tweet['place'] 





            cursor.execute(""" INSERT INTO twitter_t (ID, TIME, NAME, FULL_TEXT, HASHTAG,COORDINATE,GEO,PLACE,
                                                             FAV_COUNT,QUOTE_COUNT,REPLY_COUNT,RETWEET_COUNT,STATUS_COUNT,
                                                             SCREEN_NAME,FRIENDS_COUNT) VALUES (%s, %s, %s, %s, %s,
                                                             %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""" , (count, time.time(), [tweet['user']['name']],[text],
                                                                                                           hashtags, coords, tweet['geo'], place, 
                                                                                                           tweet['favorite_count'],tweet['quote_count'], tweet['reply_count'],
                                                                                                           tweet['retweet_count'],tweet['user']['statuses_count'], 
                                                                                                           [tweet['user']['screen_name']],tweet['user']['friends_count']))
            twitter_connection.commit()

            # Add 1 to count and if count is over 10 then stop capturing tweets
            count = count + 1
            print(count)
    
    except:
        pass
    
    time.sleep(15)

print("Done")